In [94]:
import requests
import re
from bs4 import BeautifulSoup
import os
import time

In [20]:
url = 'https://www.aptagen.com/aptamer-details/?id='

In [107]:
def saveHTML(url):
    root = '/Users/jiamingxu/Downloads/Aptahtmls/'
    urlroot = 'https://www.aptagen.com/'
    if not os.path.exists(root):
        os.mkdir(root)
    os.chdir(root)
    
    if not os.path.exists('./images'):
        os.mkdir('./images')
        
    try:
        r = requests.get(url, headers={'user-agent':'Chrome/16.0'}, timeout=30)
        r.raise_for_status
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, 'html.parser')
        htmlfile = root + soup.title.string + '.' + url.split('=')[-1] + '.html'
        
        if soup.title.string != 'Apta-Index:':
            if not os.path.exists(htmlfile) or os.stat(htmlfile).st_size == 0:
                with open(htmlfile, 'w') as f:
                    f.write(r.text)
                f.close()
                
            # Save aptamer image        
            for i in soup('img'):
                try:
                    if i['class'] == ['sequence-diagram']:
                        img = requests.get('https://www.aptagen.com'+i['src'], headers={'user-agent':'Chrome/16.0'}, timeout=30)
                        with open('./images/'+ url.split('=')[-1] + '.' +i['src'].split('/')[-1], 'wb') as imgf:
                            imgf.write(img.content)

                except:
                    continue
        
        
    except:
        print("ERROR")

    
def main():
    url = 'https://www.aptagen.com/aptamer-details/?id='
    for num in range(0,200):
        print(url+str(num))
        time.sleep(1)
        
main()

https://www.aptagen.com/aptamer-details/?id=0
https://www.aptagen.com/aptamer-details/?id=1
https://www.aptagen.com/aptamer-details/?id=2
https://www.aptagen.com/aptamer-details/?id=3
https://www.aptagen.com/aptamer-details/?id=4
https://www.aptagen.com/aptamer-details/?id=5
https://www.aptagen.com/aptamer-details/?id=6
https://www.aptagen.com/aptamer-details/?id=7
https://www.aptagen.com/aptamer-details/?id=8
https://www.aptagen.com/aptamer-details/?id=9
https://www.aptagen.com/aptamer-details/?id=10
https://www.aptagen.com/aptamer-details/?id=11
https://www.aptagen.com/aptamer-details/?id=12
https://www.aptagen.com/aptamer-details/?id=13
https://www.aptagen.com/aptamer-details/?id=14
https://www.aptagen.com/aptamer-details/?id=15
https://www.aptagen.com/aptamer-details/?id=16
https://www.aptagen.com/aptamer-details/?id=17
https://www.aptagen.com/aptamer-details/?id=18
https://www.aptagen.com/aptamer-details/?id=19
https://www.aptagen.com/aptamer-details/?id=20
https://www.aptagen.com

KeyboardInterrupt: 

In [125]:
def getHTMLText(url):
    try:
        r = requests.get(url, headers={'user-agent':'Chrome/16.0'}, timeout=30)
        r.raise_for_status
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return ""
    
def parsePage(ilt, html):
    soup = BeautifulSoup(open(html), 'html.parser')
    pat = re.compile(r'[r|f][A|G|C|T|U]p')
    seq = re.findall(pat, html)
    title = soup.title.string
    ilt.append([title, seq])
    print ("")
    
    
def printAptaSeq(ilt):
    tplt = '{:^30}\t{:^40}'
    print(tplt.format('Name','Sequence'))
    for apta in ilt:
        print(tplt.format(apta[0], apta[1]))
    

def main():
    global aptainfo
    aptainfo = []
    
    for file in os.scandir():
        if file.is_file:
            try:
                parsePage(aptainfo, re.search(r'html$',file.name).string)

            except:
                continue
            #parsePage(aptainfo, file.name)
    printAptaSeq(aptainfo)
    
main()
    































































































































































































































































































































             Name             	                Sequence                


TypeError: unsupported format string passed to list.__format__

In [127]:
len(aptainfo)

318

In [163]:
html='Apta-Index: Acetylcholine (ACh 6R).442.html'
soup = BeautifulSoup(open('Apta-Index: Acetylcholine (ACh 6R).442.html'), 'html.parser')
pat = re.compile(r'[r|f][A|G|C|T|U]p')
seq = re.findall(pat, soup)
title = soup.title.string
#ilt.append([title, seq])

TypeError: expected string or bytes-like object

In [165]:
texts = open(html).read()

In [169]:
pat = re.compile(r'[r|f|d][A|G|C|T|U]p')
seq = re.findall(pat, texts)

In [171]:
seq

['dAp',
 'dTp',
 'dCp',
 'dCp',
 'dGp',
 'dTp',
 'dCp',
 'dAp',
 'dCp',
 'dAp',
 'dCp',
 'dCp',
 'dTp',
 'dGp',
 'dCp',
 'dTp',
 'dCp',
 'dTp',
 'dCp',
 'dAp',
 'dGp',
 'dGp',
 'dGp',
 'dGp',
 'dAp',
 'dTp',
 'dCp',
 'dAp',
 'dCp',
 'dAp',
 'dTp',
 'dTp',
 'dCp',
 'dTp',
 'dTp',
 'dGp',
 'dAp',
 'dCp',
 'dGp',
 'dGp',
 'dTp',
 'dGp',
 'dTp',
 'dGp',
 'dAp',
 'dTp',
 'dAp',
 'dCp',
 'dAp',
 'dGp',
 'dTp',
 'dGp',
 'dCp',
 'dCp',
 'dTp',
 'dGp',
 'dGp',
 'dTp',
 'dGp',
 'dTp',
 'dTp',
 'dGp',
 'dGp',
 'dCp',
 'dTp',
 'dCp',
 'dCp',
 'dCp',
 'dGp',
 'dTp',
 'dAp',
 'dTp']

In [ ]:
for i in aptainfo:
    print('{:^20}\t{:^30}'.format(i[0], str(i[1])))

In [ ]:
if soup.title.string != 'Apta-Index:':
    if not os.path.exists(soup.title.string+'.html') or os.stat(soup.title.string+'.html').st_size == 0:
        with open(soup.title.string+'.html', 'w') as f:
            f.write(demo)
        f.close()
    else:
        print('ff')

In [ ]:
with open ('test.html', 'w') as ff:
    ff.write(demo)
    
ff.close()

In [ ]:
for i in soup('img'):
    try:
        if i['class']:
            img = requests.get('https://www.aptagen.com'+i['src'])
            with open(i['src'], 'wb') as imgf:
                imgf.write(img.content)
            print(i['src'])
    except:
        continue

In [12]:
def pngSave(imgpath, url):
    
    try:
        if not os.path.exists(imgpath):
            os.mkdir(imgpath)
        if not os.path.exists(imgpath):
            r = requests.get(url, headers = {'user-agent':'Chrome/16.0'})
            with open(imgpath, 'wb') as f:
                f.write(r.content)
                f.close()
                print('File saved')
        else:
            print('File exists')

    except:
        print('Failed')

In [13]:
pngSave('/Users/jiamingxu/Downloads/Aptahtmls/358.png', urlroot+'/wp-content/uploads/358.png')

Failed


In [18]:

if not os.path.exists(imgpath):
    os.mkdir(imgpath)
if not os.path.exists(imgpath):
    r = requests.get(url, headers = {'user-agent':'Chrome/16.0'})
    with open(imgpath, 'wb') as f:
        f.write(r.content)
        f.close()
        print('File saved')
else:
    print('File exists')


NameError: name 'os' is not defined

In [7]:
urlroot = 'https://www.aptagen.com/'

In [ ]:
urlroot+'/wp-content/uploads/358.png'

In [ ]:
str = 'body321389021839012<hu'
reg = /(?<=body).+(?=<hu)/;